In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pickle as pkl
from descartes import PolygonPatch

import requests
import pandas as pd
import feedparser
from bs4 import BeautifulSoup
import urllib.request
import urllib3
import json
from pandas.io.json import json_normalize
import numpy as np
from geopy.geocoders import Nominatim


# Creating CSV to go through by hand to doublecheck duplicates like I did with Cincinnati
https://chfs.ky.gov/agencies/os/oig/dhc/Pages/hcf.aspx


In [2]:
desired_counties_upper = ['BOONE','KENTON','CAMPBELL']
desired_counties_lower = ['Boone','Kenton','Campbell']

In [38]:
#Hospitals
hospitals_mf = pd.read_csv('kentucky/Hospital_Directory.csv')
hospitals_mf = hospitals_mf[hospitals_mf['COUNTY'].isin(desired_counties_lower)]
hospitals_mf.head()

,LICENSE #,NAME,ADDRESS,CITY,ZIP,COUNTY,OPERATOR NAME,Unnamed: 7,OWNER,PHONE,...,Psychiatric Geriatric (65 +),Psychiatric Adult (18-64),Psychiatric Adolescent (13-17),Psychiatric Child (12 & under),Tuberculosis,Rehabilitation,Licensed,Swing,LICENSE EXPIRATION,Unnamed: 27
23,100592,Encompass Health Rehabilitation Hospital of No...,201 Medical Village Drive,Edgewood,41017,Kenton,Jeremy,Yates,HealthSouth Northern Kentucky Rehabilitation H...,859-341-2044,...,NaN,NaN,NaN,NaN,NaN,51.0,51,N,9/30/20,NaN
30,100928,Gateway Rehabilitation Hospital,5940 Merchant Street,Florence,41042,Boone,Frank,Schneider,Vibra Rehabilitation Hospital of Florence LLC,859-426-2400,...,NaN,NaN,NaN,NaN,NaN,40.0,40,N,12/31/19,NaN
88,100500,St. Elizabeth Edgewood,1 Medical Village Dr.,Edgewood,41017,Kenton,Garren,Colvin,St Elizabeth Medical Center Inc,859-301-2000,...,NaN,NaN,NaN,NaN,NaN,NaN,518,N,12/31/19,NaN
89,100273,St. Elizabeth Florence,4900 Houston Road,Florence,41042,Boone,Bruno,Giacomuzzi,"Saint Elizabeth Medical Center, Inc.",859-2121-5220,...,NaN,NaN,NaN,NaN,NaN,NaN,231,N,10/31/20,NaN
90,100059,St. Elizabeth Ft. Thomas,85 North Grand Avenue,Ft. Thomas,41075,Campbell,John,Mitchell,"Saint Elizabeth Medical Center, Inc.",859-572-3100,...,NaN,NaN,NaN,NaN,NaN,NaN,178,N,10/31/20,NaN


In [39]:
hospitals_mf.columns
desired_columns = ['LIC TYPE','NAME','ADDRESS','CITY','ZIP','COUNTY']
hospitals_mf = hospitals_mf[desired_columns]
hospitals_mf.columns = ['TYPE', 'NAME','ADDRESS','CITY','ZIP','COUNTY']
hospitals_mf['Layer'] = 'Hospital'
hospitals_mf



,TYPE,NAME,ADDRESS,CITY,ZIP,COUNTY,Layer
23,REHAB,Encompass Health Rehabilitation Hospital of No...,201 Medical Village Drive,Edgewood,41017,Kenton,Hospital
30,REHAB,Gateway Rehabilitation Hospital,5940 Merchant Street,Florence,41042,Boone,Hospital
88,ACUTE,St. Elizabeth Edgewood,1 Medical Village Dr.,Edgewood,41017,Kenton,Hospital
89,ACUTE,St. Elizabeth Florence,4900 Houston Road,Florence,41042,Boone,Hospital
90,ACUTE,St. Elizabeth Ft. Thomas,85 North Grand Avenue,Ft. Thomas,41075,Campbell,Hospital
92,ACUTE,St. Elizabeth Medical Center,1 Medical Village Drive,Edgewood,41017,Kenton,Hospital
93,PSY,SUN Behavioral Health,820 Dolwick Drive,Erlanger,41018,Kenton,Hospital


In [40]:
hospitals_mf['TYPE'] = 'HospitalsGeneral'
hospitals_mf.loc[93, 'TYPE'] = 'HospitalsPsych'

In [41]:
#Alcohol Drug Prevention
adp_mf = pd.read_csv('kentucky/Alcohol_Drug_Prevention_Directory.csv')

adp_mf = adp_mf[adp_mf['COUNTY'].isin(desired_counties_upper)]
adp_mf['TYPE'] = 'SubstanceHelp'
desired_columns = ['TYPE','NAME','ADDRESS','CITY','ZIP','COUNTY']
adp_mf = adp_mf[desired_columns]
adp_mf.columns = ['TYPE', 'NAME','ADDRESS','CITY','ZIP','COUNTY']

adp_mf['Layer'] = 'AlcoholDrugPrevention'
adp_mf


,TYPE,NAME,ADDRESS,CITY,ZIP,COUNTY,Layer
10,SubstanceHelp,"MARK A TOENNIS, PSY, D","4150 ALEXANDRIA PIKE, SUITE 102",COLD SPRING,41076,CAMPBELL,AlcoholDrugPrevention
14,SubstanceHelp,NORTH KEY COMMUNITY CARE REGIONAL PREVENTION CTR,7459 BURLINGTON PIKE,FLORENCE,41042,BOONE,AlcoholDrugPrevention


In [42]:
#Alcohol and Other Drug Entity and Behavioral Health Service Organization

aode_bhso_mf = pd.read_csv('kentucky/AODE_BHSO_Directory.csv')

aode_bhso_mf = aode_bhso_mf[aode_bhso_mf['COUNTY'].isin(desired_counties_upper)]

desired_columns = ['TYPE','NAME','ADDRESS','CITY','ZIP','COUNTY']
aode_bhso_mf = aode_bhso_mf[desired_columns]
aode_bhso_mf['Layer'] = 'AODE_BHSO'

aode_bhso_mf.reset_index(inplace=True)
aode_bhso_mf.drop(['index'], axis=1, inplace=True)

aode_bhso_mf

,TYPE,NAME,ADDRESS,CITY,ZIP,COUNTY,Layer
0,Alcohol and Other Drug Entity Facility,ADDICTION SERVICES COUNCIL,20 WEST 18TH STREET,COVINGTON,41011,KENTON,AODE_BHSO
1,Alcohol and Other Drug Entity Facility,ALCOHOL SERVICES OF KENTUCKY,"7505 SUSSEX DRIVE, SUITE 214",FLORENCE,41042,BOONE,AODE_BHSO
2,Alcohol and Other Drug Entity Facility,AWARENESS & DISCOVERY,2 DORTHA AVENUE,FLORENCE,41042,BOONE,AODE_BHSO
3,Alcohol and Other Drug Entity Facility,COMMONWEALTH SUBSTANCE ABUSE SPECIALISTS,"7000 HOUSTON RD, BLDG 400, SUITE 43",FLORENCE,41042,BOONE,AODE_BHSO
4,Alcohol and Other Drug Entity Facility,"CONNECT COUNSELING, LLC",8333 ALEXANDRIA PIKE,ALEXANDRIA,41001,CAMPBELL,AODE_BHSO
5,Alcohol and Other Drug Entity Facility,COVINGTON TREATMENT CENTER,1450 MADISON AVENUE,COVINGTON,41011,KENTON,AODE_BHSO
6,Alcohol and Other Drug Entity Facility,MENTAL HEALTH AMERICA OF NORTHERN KENTUCKY,912 SCOTT BOULEVARD,COVINGTON,41011,KENTON,AODE_BHSO
7,Alcohol and Other Drug Entity Facility,NECCO,"75 CAVALIER BOULEVARD, SUITE 110",FLORENCE,41042,BOONE,AODE_BHSO
8,Alcohol and Other Drug Entity Facility,NKY MED CLINIC LLC,1717 MADISON AVENUE,COVINGTON,41011,KENTON,AODE_BHSO
9,Alcohol and Other Drug Entity Facility,NORTH KEY COMMUNITY CARE,503 FARRELL DRIVE,COVINGTON,41012,KENTON,AODE_BHSO


In [43]:
aode_bhso_mf.loc[0:15,'TYPE'] = 'SubstanceHelp'
aode_bhso_mf.loc[17:18,'TYPE'] = 'Mental'
aode_bhso_mf.loc[19:25,'TYPE'] = 'SubstanceHelp'
aode_bhso_mf.loc[26:33,'TYPE'] = 'Mental'


aode_bhso_mf.loc[16,'TYPE'] = 'ChildFacility'
aode_bhso_mf.loc[20,'TYPE'] = 'ChildFacility'
aode_bhso_mf.loc[23:24,'TYPE'] = 'ChildFacility'
aode_bhso_mf.loc[27,'TYPE'] = 'ChildFacility'
aode_bhso_mf.loc[30:31,'TYPE'] = 'ChildFacility'

In [9]:
#Long Term Care
long_mf = pd.read_csv('kentucky/Long_Term_Care_Directory.csv')

long_mf = long_mf[long_mf['County'].isin(desired_counties_upper)]

long_mf['TYPE'] = 'NursingHome'

desired_columns = ['TYPE','Facility Name','Address Line 1','City','Zip Code','County']
long_mf = long_mf[desired_columns]
long_mf.columns = ['TYPE', 'NAME', 'ADDRESS','CITY','ZIP','COUNTY']
long_mf['Layer'] = 'LongTerm'

long_mf

,TYPE,NAME,ADDRESS,CITY,ZIP,COUNTY,Layer
11,NursingHome,"BOONESPRING TRANSITIONAL CARE CENTER, LLC",10250 US HIGHWAY 42,UNION,41091,BOONE,LongTerm
21,NursingHome,BRIDGE POINT CENTER,7300 WOODSPOINT DRIVE,FLORENCE,41042,BOONE,LongTerm
30,NursingHome,CARMEL MANOR,100 CARMEL MANOR ROAD,FORT THOMAS,41075,CAMPBELL,LongTerm
52,NursingHome,"COLDSPRING TRANSITIONAL CARE CENTER, LLC",300 PLAZA DRIVE,COLD SPRING,41076,CAMPBELL,LongTerm
73,NursingHome,EMERALD TRACE,3802 TURKEYFOOT RD,ELSMERE,41018,KENTON,LongTerm
78,NursingHome,FLORENCE PARK CARE CENTER,6975 BURLINGTON PIKE,FLORENCE,41042,BOONE,LongTerm
113,NursingHome,HIGHLANDSPRING OF FT THOMAS,960 HIGHLAND AVENUE,FORT THOMAS,41075,CAMPBELL,LongTerm
155,NursingHome,MADONNA MANOR,2344 AMSTERDAM ROAD,VILLA HILLS,41017,KENTON,LongTerm
219,NursingHome,ROSEDALE GREEN,4250 GLENN AVENUE,COVINGTON,41015,KENTON,LongTerm
260,NursingHome,ST ELIZABETH FLORENCE SNF,4900 HOUSTON ROAD,FLORENCE,41042,BOONE,LongTerm


In [44]:
#Miscellaneous
med_mf = pd.read_csv('kentucky/Miscellaneous_Directory.csv')


med_mf = med_mf[med_mf['COUNTY'].isin(desired_counties_upper)]
desired_columns = ['FACILTY\nTYPE','NAME','ADDRESS','CITY','ZIP','COUNTY']
med_mf = med_mf[desired_columns]
med_mf.columns = ['TYPE', 'NAME', 'ADDRESS','CITY','ZIP','COUNTY']
med_mf['Layer'] = 'Miscellaneous'

med_mf = med_mf[~med_mf.TYPE.str.contains("ADULT DAY HEALTH")]
med_mf = med_mf[~med_mf.TYPE.str.contains("(HHA)")]
med_mf = med_mf[~med_mf.NAME.str.contains("DIALYSIS")]
med_mf = med_mf[~med_mf.TYPE.str.contains("MRDD")]
med_mf = med_mf[~med_mf.TYPE.str.contains("X-RAY")]

med_mf.sort_values(by=['TYPE'], inplace=True)
med_mf.reset_index(inplace=True)
med_mf.drop(['index'], axis=1, inplace=True)


med_mf.drop([8, 10,13, 18], inplace=True)
med_mf.reset_index(inplace=True)
med_mf.drop(['index'], axis=1, inplace=True)
med_mf

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if sys.path[0] == '':


,TYPE,NAME,ADDRESS,CITY,ZIP,COUNTY,Layer
0,AMBULATORY SURGICAL CENTER,"CENTER FOR SURGICAL CARE,THE",7575 US 42,FLORENCE,41042,BOONE,Miscellaneous
1,AMBULATORY SURGICAL CENTER,"DAYBREAK OPERATIONS, LLC",133 BARNWOOD DRIVE,EDGEWOOD,41017,KENTON,Miscellaneous
2,AMBULATORY SURGICAL CENTER,ST ELIZABETH PHYSICIANS ENDOSCOPY CENTER,"340 THOMAS MORE PARKWAY, SUITE 160 B",CRESTVIEW HILLS,41017,KENTON,Miscellaneous
3,AMBULATORY SURGICAL CENTER,TRI STATE DIGESTIVE DISORDER CENTER,425 CENTRE VIEW BOULEVARD,CRESTVIEW HILLS,41017,KENTON,Miscellaneous
4,AMBULATORY SURGICAL CENTER-LICENSED ONLY,SURGERY CENTER CRESTVIEW,2845 CHANCELLOR DRIVE,CRESTVIEW HILLS,41017,KENTON,Miscellaneous
5,COMMUNITY MENTAL HEALTH CENTERS- LIC ONLY,NORTHKEY COMMUNITY CARE,503 FARRELL DRIVE,COVINGTON,41012,KENTON,Miscellaneous
6,END STAGE RENAL DISEASE FACILITIES,FRESENIUS MEDICAL CARE BOONE COUNTY,7205 DIXIE HIGHWAY,FLORENCE,41042,BOONE,Miscellaneous
7,END STAGE RENAL DISEASE FACILITIES,FRESENIUS MEDICAL CARE EDGEWOOD,210 THOMAS MOORE PARKWAY,EDGEWOOD,41017,KENTON,Miscellaneous
8,FREESTANDING OR MOBILE TECHNOLOGY,ST ELIZABETH MEDICAL CENTER AMBULATORY CARE CE...,1500 JAMES SIMPSON WAY,COVINGTON,41011,KENTON,Miscellaneous
9,FREESTANDING OR MOBILE TECHNOLOGY,ST ELIZABETH INDEPENDENCE,135 COURTHOUSE CROSSING,INDEPENDENCE,41051,KENTON,Miscellaneous


In [45]:
med_mf.loc[0:4,'TYPE'] = 'Surgery'
med_mf.loc[5,'TYPE'] = 'Mental'
med_mf.loc[6:7,'TYPE'] = 'MedicalFacility'
med_mf.loc[8:9,'TYPE'] = 'UrgentCare'
med_mf.loc[10,'TYPE'] = 'Hospice'
med_mf.loc[13:14,'TYPE'] = 'MedicalFacility'
med_mf.loc[11:12,'TYPE'] = 'ChildFacility'
med_mf.loc[15,'TYPE'] = 'ChildFacility'


In [46]:
#Physician_Office_Laboratories_Directory
pol_mf = pd.read_csv('kentucky/Physician_Office_Laboratories_Directory.csv')

pol_mf = pol_mf[pol_mf['COUNTY'].isin(desired_counties_upper)]

pol_mf['TYPE'] = 'Unknown'

desired_columns = ['TYPE','NAME','ADDRESS','CITY','ZIP','COUNTY']
pol_mf = pol_mf[desired_columns]

pol_mf['Layer'] = 'PhysicianOffices'

pol_mf.sort_values(by=['ADDRESS'], inplace=True)
pol_mf.reset_index(inplace=True)
pol_mf.drop(['index'], axis=1, inplace=True)

pol_mf.drop([16,17, 18, 99, 101, 114, 119, 124, 130], inplace=True)
pol_mf.reset_index(inplace=True)
pol_mf.drop(['index'], axis=1, inplace=True)

pol_child_mf = pol_mf[pol_mf.NAME.str.contains("PEDIATRICS")]
pol_child_mf1 = pol_mf[pol_mf.NAME.str.contains("PEDIATRIC")]
pol_child_mf2 = pol_mf[pol_mf.NAME.str.contains("FAMILY")]

pol_women_mf = pol_mf[pol_mf.NAME.str.contains("WOMEN")]
pol_women_mf1 = pol_mf[pol_mf.NAME.str.contains("OB/GYN")]
pol_women_mf2 = pol_mf[pol_mf.NAME.str.contains("WOMEN'S")]
pol_mf = pol_mf[~pol_mf.NAME.str.contains("WOMEN'S")]

medical = pol_mf[pol_mf.NAME.str.contains("ST ELIZABETH PHYSICIANS")]
urgent = pol_mf[pol_mf.NAME.str.contains("URGENT CARE")]
pain = pol_mf[pol_mf.NAME.str.contains("PAIN")]
physical = pol_mf[pol_mf.NAME.str.contains("BEACON ORTHOPAEDIC & SPORTS MEDICINE")]

pol_mf = pol_mf[~pol_mf.NAME.str.contains("PEDIATRICS")]
pol_mf = pol_mf[~pol_mf.NAME.str.contains("PEDIATRIC")]
pol_mf = pol_mf[~pol_mf.NAME.str.contains("FAMILY")]
pol_mf = pol_mf[~pol_mf.NAME.str.contains("WOMEN")]
pol_mf = pol_mf[~pol_mf.NAME.str.contains("OB/GYN")]

pol_mf = pol_mf[~pol_mf.NAME.str.contains("ST ELIZABETH PHYSICIANS")]
pol_mf = pol_mf[~pol_mf.NAME.str.contains("URGENT CARE")]
pol_mf = pol_mf[~pol_mf.NAME.str.contains("PAIN")]
pol_mf = pol_mf[~pol_mf.NAME.str.contains("BEACON ORTHOPAEDIC & SPORTS MEDICINE")]
pol_mf.drop([34], inplace=True)

pol_mf.reset_index(inplace=True)
pol_mf.drop(['index'], axis=1, inplace=True)
pol_mf.drop([33], inplace=True)
pol_mf

,TYPE,NAME,ADDRESS,CITY,ZIP,COUNTY,Layer
0,Unknown,TCHMA ATKINS AND PEARCE,1 BRAID WAY,COVINGTON,41017,KENTON,PhysicianOffices
1,Unknown,"WOMANKIND MIDWIFERY, PLLC",10 YOUELL STREET,FLORENCE,41042,BOONE,PhysicianOffices
2,Unknown,SHEARER DAVID MD,10060 DEMIA WAY,FLORENCE,41042,BOONE,PhysicianOffices
3,Unknown,JIBRINI MD ASSOCIATES PLLC,103 LANDMARK DRIVE STE 200,BELLEVUE,41073,CAMPBELL,PhysicianOffices
4,Unknown,"HAROLD T PRETORIUS, MD","103 LANDMARK DRIVE, SUITE 365",BELLEVUE,41073,CAMPBELL,PhysicianOffices
5,Unknown,MADISON PIKE HEALTH CENTER,1045 EASTON DRIVE,FT WRIGHT,41017,KENTON,PhysicianOffices
6,Unknown,ST ELIZABETH REGIONAL DIABETES CENTER,"1500 JAMES SIMPSON WAY, SUITE 301",COVINGTON,41011,KENTON,PhysicianOffices
7,Unknown,OTRI-MED CORPORATION,162 BARNWOOD DRIVE,EDGEWOOD,41017,KENTON,PhysicianOffices
8,Unknown,KENWOOD DERMATOLOGY,175 FRENCH STREET,COLD SPRING,41076,CAMPBELL,PhysicianOffices
9,Unknown,PRIMARY EYE CARE ASSOCIATES,"1821 FLORENCE PIKE, SUITE 1",BURLINGTON,41005,BOONE,PhysicianOffices


In [47]:
pol_women_mf.append(pol_women_mf1, ignore_index=True)
pol_women_mf.append(pol_women_mf2, ignore_index=True)
pol_women_mf.append(pol_mf.loc[1], ignore_index=True)
pol_women_mf.append(pol_mf.loc[11], ignore_index=True)
pol_women_mf['TYPE'] = 'WomenClinic'
pol_mf.drop([1,11], inplace=True)
pol_women_mf

,TYPE,NAME,ADDRESS,CITY,ZIP,COUNTY,Layer
16,WomenClinic,ST ELIZABETH PHYSICIANS WOMEN'S HEALTH FT THOMAS,1400 N GRAND AVENUE,NEWPORT,41071,CAMPBELL,PhysicianOffices
37,WomenClinic,ST ELIZABETH PHYSICIANS WOMEN'S HEALTH EDGEWOOD,20 MEDICAL VILLAGE DR SUITE 308,EDGEWOOD,41017,KENTON,PhysicianOffices
101,WomenClinic,ST ELIZABETH PHYSICIANS WOMEN'S HEALTH FLORENCE -,6907 BURLINGTON PIKE,FLORENCE,41042,BOONE,PhysicianOffices
103,WomenClinic,ALL WOMEN PLLC,"7000 HUSTON ROAD, SUITE 19",FLORENCE,41042,BOONE,PhysicianOffices
107,WomenClinic,ST ELIZABETH PHYSICIANS WOMEN'S HEALTH,7370 TURFWAY ROAD,FLORENCE,41042,BOONE,PhysicianOffices
112,WomenClinic,ST ELIZABETH PHYSICIANS WOMEN'S HEALTH,"7370 TURFWAY ROAD, SUITE 390",FLORENCE,41042,BOONE,PhysicianOffices


In [48]:
pol_child_mf.append(pol_child_mf1, ignore_index=True)
pol_child_mf.append(pol_child_mf2, ignore_index=True)
pol_child_mf['TYPE'] = 'ChildFacility'
pol_child_mf

,TYPE,NAME,ADDRESS,CITY,ZIP,COUNTY,Layer
2,ChildFacility,"WHOLE CHILD PEDIATRICS,THE",10032 DEMIA WAY,FLORENCE,41042,BOONE,PhysicianOffices
63,ChildFacility,CRESTVIEW HILLS PEDIATRICS,334 THOMAS MORE PKWY SUITE 100,CRESTVIEW HILLS,41017,KENTON,PhysicianOffices
83,ChildFacility,RIVER HILLS PEDIATRICS,"525 ALEXANDRIA PIKE, SUITE 320",SOUTHGATE,41071,CAMPBELL,PhysicianOffices
90,ChildFacility,PRIMARY PEDIATRICS PSC,59 CAVALIER BOULEVARD SUITE 330,FLORENCE,41042,BOONE,PhysicianOffices
113,ChildFacility,PEDIATRICS OF FLORENCE PSC,7409 US 42,FLORENCE,41042,BOONE,PhysicianOffices
122,ChildFacility,"UNION PEDIATRICS, PSC",8667 US HIGHWAY 42 STE 300,UNION,41091,BOONE,PhysicianOffices


In [49]:
medical.drop([14,47, 64, 98, 108,111], inplace=True)
medical['TYPE'] = 'MedicalFacility'
medical.loc[62, 'TYPE']='Mental'

In [50]:
urgent['TYPE'] = 'UrgentCare'

In [51]:
pain['TYPE'] = 'PainManagement'

In [52]:
physical['TYPE'] = 'PhysicalTherapy'

In [53]:
pol_mf['TYPE'] = 'MedicalFacility'

In [54]:
#ALL FACILITIES (PLEASE NOTE THERE WERE NO PHYSICAL THERAPY FACILITIES LISTED)
facilities = pd.concat([hospitals_mf, aode_bhso_mf, long_mf, med_mf, pol_women_mf, pol_child_mf, medical, urgent, pain, physical, pol_mf],axis=0,
                       ignore_index=True,sort=False)
facilities.to_csv(r'kentucky/MedicalFacilitiesIn12miKY.csv')

In [21]:
facilities.sort_values(by=['ADDRESS'])

,TYPE,NAME,ADDRESS,CITY,ZIP,COUNTY,Layer
125,MedicalFacility,TCHMA ATKINS AND PEARCE,1 BRAID WAY,COVINGTON,41017,KENTON,PhysicianOffices
2,HospitalsGeneral,St. Elizabeth Edgewood,1 Medical Village Dr.,Edgewood,41017,Kenton,Hospital
5,HospitalsGeneral,St. Elizabeth Medical Center,1 Medical Village Drive,Edgewood,41017,Kenton,Hospital
43,NursingHome,CARMEL MANOR,100 CARMEL MANOR ROAD,FORT THOMAS,41075,CAMPBELL,LongTerm
79,ChildFacility,"WHOLE CHILD PEDIATRICS,THE",10032 DEMIA WAY,FLORENCE,41042,BOONE,PhysicianOffices
126,MedicalFacility,SHEARER DAVID MD,10060 DEMIA WAY,FLORENCE,41042,BOONE,PhysicianOffices
41,NursingHome,"BOONESPRING TRANSITIONAL CARE CENTER, LLC",10250 US HIGHWAY 42,UNION,41091,BOONE,LongTerm
127,MedicalFacility,JIBRINI MD ASSOCIATES PLLC,103 LANDMARK DRIVE STE 200,BELLEVUE,41073,CAMPBELL,PhysicianOffices
118,PainManagement,GREATER CINCINNATI PAIN MANAGEMENT CENTERS,"103 LANDMARK DRIVE, SUITE 110",BELLEVUE,41073,CAMPBELL,PhysicianOffices
128,MedicalFacility,"HAROLD T PRETORIUS, MD","103 LANDMARK DRIVE, SUITE 365",BELLEVUE,41073,CAMPBELL,PhysicianOffices


In [55]:

facilities.drop([5,128,130,135,136,138,139,34,72,38,22,150,121,151,30,105,166,78,174],inplace=True)

facilities.reset_index(inplace=True)
facilities.drop(['index'], axis=1, inplace=True)

facilities.to_csv(r'kentucky/MedicalFacilitiesIn12miKY.csv')

In [56]:
#Based on looking at a map
np.unique(facilities['CITY'])
desired_cities = ['BELLEVUE','COVINGTON','CRESCENT SPRINGS', 'CRESTVIEW HILLS', 'EDGEWOOD',
                  'ERLANGER', 'Edgewood', 'Erlanger', 'FORT MITCHELL',
                  'FORT THOMAS', 'FT WRIGHT', 'Ft. Thomas', 'HEBRON',
                  'NEWPORT', 'SOUTHGATE','VILLA HILLS']

facilities = facilities[facilities['CITY'].isin(desired_cities)]
print(len(facilities))

97


In [68]:
from geopy.geocoders import Nominatim



def facility_address(df, city):
    """takes dataframe and returns the latitude and longitude and makes geopandas dataframe 
    Args:
        df (dataframe) - dataframe to get addresses from
    Returns:
        geo_df - new geopandas dataframe that has the latitude and longitude of the points
    """
    new_df = pd.DataFrame()
    crs = {'init': 'epsg:4326'}
    lats = []
    longs = []
    c_df = df[df['CITY'] == city]
    c_df.reset_index(inplace=True)
    c_df.drop(['index'], axis=1, inplace=True)
    
    if city == 'BELLEVUE':
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Bellevue Kentucky")
        for i in range(len(c_df)):
            address = c_df.loc[i,'ADDRESS']
            if address == '103 LANDMARK DRIVE, SUITE 110':
                address = '103 LANDMARK DRIVE, STE 110'
            if address == '119 FAIRFIELD AVENUE, SUITE R102':
                address = '119 FAIRFIELD AVENUE, STE 102'
            if address == '103 LANDMARK DRIVE, SUITE 365':
                address = '103 LANDMARK DRIVE STE 365'
            #print(address)
            location = geolocator.geocode(address)
            #print(location)
            lat = location.latitude
            long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df
        
    elif city == 'COVINGTON':
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Covington Kentucky")
        for i in range(len(c_df)):
            address = c_df.loc[i,'ADDRESS']
            print(address)
            if address == '434 SCOTT BOULEVARD, 2ND FLOOR':
                address = '434 SCOTT BOULEVARD'
            if address == '303 COURT STREET, 1ST FLOOR':
                address = '303 COURT STREET'
            location = geolocator.geocode(address)
            print(location)
            if address == '200 HOME ROAD':
                lat = float(39.085836)
                long = float(-84.539347)
            elif address == '1500 JAMES SIMPSON WAY' or address == '1500 JAMES SIMPSON JR WAY' or address == '1500 JAMES SIMPSON WAY, SUITE 301':
                lat = float(39.071435)
                long = float(-84.518014)
            else:
                lat = location.latitude
                long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df
        
    elif city == 'CRESCENT SPRINGS':
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Crescent Springs Kentucky")
        for i in range(len(c_df)):
            address = c_df.loc[i,'ADDRESS']
            #print(address)
            location = geolocator.geocode(address)
            #print(location)
            lat = location.latitude
            long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df
        
    elif city == 'CRESTVIEW HILLS':
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Crestview Kentucky")
        for i in range(len(c_df)):
            address = c_df.loc[i,'ADDRESS']
            #print(address)
            if address == '320 THOMAS MORE PARKWAY SUITE 202' or address == '320 THOMAS MORE PARKWAY, SUITE 202':
                address = '320 THOMAS MORE PARKWAY STE 202'
            if address == '320 THOMAS MORE PKWY, SUITE 201':
                address = '320 THOMAS MORE PKWY STE 201'
            if address == '334 THOMAS MORE PKWY SUITE 100':
                address = '334 THOMAS MORE PKWY STE 100'
            if address == '334 THOMAS MORE PKWY SUITE 200':
                address = '334 THOMAS MORE PKWY STE 200'
            if address == '340 THOMAS MORE PARKWAY, SUITE 160 B':
                address = '340 THOMAS MORE PARKWAY STE 160'
            if address == '340 THOMAS MORE PARKWAY, SUITE 260':
                address = '340 THOMAS MORE PARKWAY STE 260'
            if address == '350 THOMAS MORE PARKWAY SUITE 280':
                address = '350 THOMAS MORE PARKWAY STE 280'
            if address == '500 THOMAS MOORE PARKWAY STE 1':
                address = '500 THOMAS MORE PARKWAY STE 1'
            if address == '651 CENTRE VIEW':
                address = '651 CENTRE VIEW BLVD'
            if address == '34O THOMAS MORE PARKWAY, SUITE 160B':
                address = '340 THOMAS MORE PARKWAY STE 160'
            location = geolocator.geocode(address)
            #print(location)
            if address == '2765 CHAPEL PLACE STE 250':
                lat = float(39.024193)
                long = float(-84.562961)
            else:
                lat = location.latitude
                long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df
    
    elif city == 'EDGEWOOD' or city == 'Edgewood':
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Edgewood Kentucky")
        for i in range(len(c_df)):
            #print(c_df.loc[i,'City'])
            address = c_df.loc[i,'ADDRESS']
            if address == '210 THOMAS MOORE PARKWAY':
                address = '210 THOMAS MORE PARKWAY'
            if address == '3005 DIXIE, SUITE 130':
                address = '3005 DIXIE #130'
            if address == '20 MEDICAL VILLAGE DRIVE SUITE 355':
                address = '20 MEDICAL VILLAGE DRIVE #355'
            if address == '20 MEDICAL VILLAGE DRIVE, SUITE 200':
                address = '20 MEDICAL VILLAGE DRIVE #200'
            if address == '20 MEDICAL VILLAGE DR SUITE 308':
                address = '20 MEDICAL VILLAGE DR #308'
            if address == '830 THOMAS MORE PARKWAY SUITE #202':
                address = '830 THOMAS MORE PARKWAY #202'
            #print(address)
            location = geolocator.geocode(address)
            #print(location)
            if address == '483 SOUTH LOOP DRIVE':
                lat = float(39.009270)
                long = float(-84.562784)
            elif address == '580 SOUTH LOOP ROAD, SUITE 200':
                lat = float(39.012048)
                long = float(-84.559416)
            else:
                lat = location.latitude
                long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df
        
    elif city == 'ERLANGER' or city == 'Erlanger':
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Erlanger Kentucky")
        for i in range(len(c_df)):
            address = c_df.loc[i,'ADDRESS']
            #print(address)
            if address == '510 GRAVES AVENUE, SUITE 206':
                address = '510 GRAVES AVENUE STE 206'
            location = geolocator.geocode(address)
            #print(location)
            if address == '2990 RIGGS AVENUE':
                lat = float(39.0306383)
                long = float(-84.5971703)
            else:
                lat = location.latitude
                long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df
    
    elif city == 'FORT MITCHELL':
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Fort Mitchell Kentucky")
        for i in range(len(c_df)):
            address = c_df.loc[i,'ADDRESS']
            #print(address)
            location = geolocator.geocode(address)
            #print(location)
            if address == '2300 CHAMBER CENTER DRIVE, SUITE 200':
                lat = float(39.0443954)
                long = float(-84.5718202)
            else:
                lat = location.latitude
                long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df
        
    elif city == 'FORT THOMAS' or city == 'Ft. Thomas':
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Fort Thomas Kentucky")
        for i in range(len(c_df)):
            address = c_df.loc[i,'ADDRESS']
            #print(address)
            location = geolocator.geocode(address)
            #print(location)
            lat = location.latitude
            long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df
    
    elif city == 'FT WRIGHT':
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Fort Wright Kentucky")
        for i in range(len(c_df)):
            address = c_df.loc[i,'ADDRESS']
            #print(address)
            if address == '1045 EASTON DRIVE':
                address = '1045 EATON DRIVE'
            if address == '1955 DIXIE HIGHWAY, SUITE M' or address == '1955 DIXIE HWY STE F':
                address = '1955 DIXIE HIGHWAY'
            location = geolocator.geocode(address)
            #print(location)
            lat = location.latitude
            long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df
        
    elif city == 'HEBRON':
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Hebron Kentucky")
        for i in range(len(c_df)):
            address = c_df.loc[i,'ADDRESS']
            #print(address)
            if address == '2093 MEDICAL ARTS DRIVE, 1ST FLOOR':
                address = '2093 MEDICAL ARTS DRIVE'
            location = geolocator.geocode(address)
            #print(location)
            lat = location.latitude
            long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df
        
    elif city == 'NEWPORT':
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Newport Kentucky")
        for i in range(len(c_df)):
            address = c_df.loc[i,'ADDRESS']
            location = geolocator.geocode(address)
            #print(address)
            #print(location)
            if address == '7341 E ALEXANDRIA PIKE':
                lat = float(38.991852)
                long = float(-84.397925)
            elif address == '1400 GRAND AVENUE' or address == '1400 GRAND AVE' or address == '1400 N GRAND AVENUE':
                lat = float(39.080640)
                long = float(-84.469422)
            else:
                lat = location.latitude
                long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df
    
    elif city == 'SOUTHGATE':
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Southgate Kentucky")
        for i in range(len(c_df)):
            address = c_df.loc[i,'ADDRESS']
            #print(address)
            if address == '525 ALEXANDRIA PIKE, SUITE 230':
                address = '525 ALEXANDRIA PIKE STE 230'
            if address == '525 ALEXANDRIA PIKE, SUITE 320':
                address = '525 ALEXANDRIA PIKE STE 320'
            location = geolocator.geocode(address)
            #print(location)
            lat = location.latitude
            long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df

    else:
        geolocator = Nominatim(user_agent="my_app", format_string="%s, Villa Hills Kentucky")
        for i in range(len(c_df)):
            address = c_df.loc[i,'ADDRESS']
            #print(address)
            location = geolocator.geocode(address)
            #print(location)
            lat = location.latitude
            long = location.longitude
            lats.append(lat)
            longs.append(long)
        c_df['Lat'] = lats
        c_df['Long'] = longs
        new_df = c_df
    geometry = [Point(xy) for xy in zip(new_df['Long'],new_df['Lat'])]
    geo_df = gpd.GeoDataFrame(new_df, crs = crs, geometry = geometry)
    return(geo_df)
                
                
                

In [65]:
geolocator = Nominatim(user_agent="my_app", format_string="%s, Crestview Hills Kentucky")
address = '340 THOMAS MORE PARKWAY STE 160'
location = geolocator.geocode(address)
location

Location(340, Thomas More Parkway, Crestview Hills, Kenton County, Kentucky, 41017, United States, (39.0239077556644, -84.5684551703933, 0.0))

In [66]:
bellevue_geo = facility_address(facilities, 'BELLEVUE')

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [59]:
#Facilities in COVINGTON
covington_geo = facility_address(facilities, 'COVINGTON')

20 WEST 18TH STREET
20, West 18th Street, Wallace Woods, Latonia, Covington, Kenton County, Kentucky, 41011, United States
1450 MADISON AVENUE
1450, Madison Avenue, Eastside, Covington, Kenton County, Kentucky, 41011, United States
912 SCOTT BOULEVARD
912, Scott Boulevard, Eastside, Covington, Kenton County, Kentucky, 41011, United States of America
1717 MADISON AVENUE
1717, Madison Avenue, Eastside, Covington, Kenton County, Kentucky, 41011, United States
503 FARRELL DRIVE
Farrell Drive, Latonia, Covington, Kenton County, Kentucky, 41011, United States of America
3000 DECKER CRANE LANE
Decker Crane Lane, Covington, Kenton County, Kentucky, 41015, United States
1717 MADISON AVENUE
1717, Madison Avenue, Eastside, Covington, Kenton County, Kentucky, 41011, United States
200 HOME ROAD
None
4339 WINSTON AVENUE
Winston Avenue, Latonia, Covington, Kenton County, Kentucky, 41015, United States
434 SCOTT BOULEVARD
434, Scott Boulevard, Latonia, Covington, Kenton County, Kentucky, 41011, United

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [60]:
crescentSprings_geo = facility_address(facilities, 'CRESCENT SPRINGS')

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [69]:
crestviewHills_geo = facility_address(facilities, 'CRESTVIEW HILLS')

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [70]:
#Facilities in 'EDGEWOOD', 'Edgewood'
edgewood1_geo = facility_address(facilities, 'EDGEWOOD')
edgewood2_geo = facility_address(facilities, 'Edgewood')
edgewood_geo = pd.concat([edgewood1_geo, edgewood2_geo],axis=0,
                       ignore_index=True,sort=False)

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [71]:
erlanger1_geo = facility_address(facilities, 'ERLANGER')
erlanger2_geo = facility_address(facilities, 'Erlanger')
erlanger_geo = pd.concat([erlanger1_geo, erlanger2_geo], axis=0,
                       ignore_index=True,sort=False)

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [72]:
ftmitchell_geo = facility_address(facilities, 'FORT MITCHELL')

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:192: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [73]:
#Facilities for 'FORT THOMAS', 'Ft. Thomas',
ftthomas1_geo = facility_address(facilities, 'FORT THOMAS')
ftthomas2_geo = facility_address(facilities, 'Ft. Thomas')
ftthomas_geo = pd.concat([ftthomas1_geo, ftthomas2_geo],axis=0,
                       ignore_index=True,sort=False)

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [74]:
ftwright_geo = facility_address(facilities, 'FT WRIGHT')

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
hebron_geo = facility_address(facilities, 'HEBRON')

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:243: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [76]:
#Facilities for NEWPORT
newport_geo = facility_address(facilities, 'NEWPORT')

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [77]:
southgate_geo = facility_address(facilities, 'SOUTHGATE')

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:285: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [78]:
villahills_geo = facility_address(facilities, 'VILLA HILLS')

/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maddiewarndorf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [79]:
facilities_geo = pd.concat([bellevue_geo, crescentSprings_geo, crestviewHills_geo,
                            covington_geo, edgewood_geo, erlanger_geo, ftmitchell_geo, 
                            ftthomas_geo, ftwright_geo, hebron_geo, newport_geo,
                            southgate_geo, villahills_geo],axis=0,
                       ignore_index=True,sort=False)
facilities_geo.to_csv(r'kentucky/MedicalFacilitiesIn12miKYgeo.csv')
facilities_geo.to_csv(r'DataFiles/FacilityData/MedicalFacilitiesIn12miKYgeo.csv')

In [80]:
facilities_geo.head()

,TYPE,NAME,ADDRESS,CITY,ZIP,COUNTY,Layer,Lat,Long,geometry
0,MedicalFacility,ST ELIZABETH PHYSICIANS,"119 FAIRFIELD AVENUE, SUITE R102",BELLEVUE,41073,CAMPBELL,PhysicianOffices,39.103593,-84.487308,POINT (-84.48731 39.10359)
1,PainManagement,GREATER CINCINNATI PAIN MANAGEMENT CENTERS,"103 LANDMARK DRIVE, SUITE 110",BELLEVUE,41073,CAMPBELL,PhysicianOffices,39.099644,-84.485983,POINT (-84.48598 39.09964)
2,MedicalFacility,JIBRINI MD ASSOCIATES PLLC,103 LANDMARK DRIVE STE 200,BELLEVUE,41073,CAMPBELL,PhysicianOffices,39.099644,-84.485983,POINT (-84.48598 39.09964)
3,MedicalFacility,"TAKE CARE HEALTH KENTUCKY, PSC",606 BUTTERMILK PIKE,CRESCENT SPRINGS,41017,KENTON,PhysicianOffices,39.052425,-84.577781,POINT (-84.57778 39.05243)
4,SubstanceHelp,ST ELIZABETH PHYSICIAN'S JOURNEY RECOVERY CENTER,351 CENTRE VIEW BLVD,CRESTVIEW HILLS,41017,KENTON,AODE_BHSO,39.023186,-84.563881,POINT (-84.56388 39.02319)


In [81]:
with open('DataFiles/MedicalFacilitiesIn12miKYgeo.pkl', 'wb') as pkl_file:
    pkl.dump(facilities_geo, pkl_file)